In [1]:
!pip install langchain_community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.7 MB/s eta 0:00:00


In [2]:
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
caminho_pasta = '/content/drive/Shareddrives/LLMs/Datasets/CNPJ/'

#### Funções Auxiliares

In [5]:
def get_schema(db_path):
  connection_string="sqlite:///"+db_path
  db = SQLDatabase.from_uri(connection_string, sample_rows_in_table_info=0)
  return str(db.table_info)

In [6]:
def informacoes(bd):
  conn = sqlite3.connect(caminho_pasta+bd)
  cursor = conn.cursor()

  cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

  tables = cursor.fetchall() # obtenha todos os nomes das tabelas

  for table in tables: # para cada tabela, imprima suas informações
      print("\nTable:", table[0],end='')
      table_name = table[0]
      cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
      count = cursor.fetchone()[0]
      print(f"- {count} linhas.")

      cursor.execute("PRAGMA table_info('{}')".format(table[0])) # execute uma consulta para obter informações sobre as colunas da tabela

      columns = cursor.fetchall() #retorna uma tupla com informações de cada coluna
      for column in columns: #cada colums = tupla = (Indice, Nome da Coluna, Tipo de Dados, Indicador de Não Nulo, Valor Padrão,  Chave Primária )
          print("*",column[1], "-", column[2], "  (PK=", column[5] == 1,")")

  cursor.close()
  conn.close()

In [7]:
def consultar(bd, query):
    try:
        conn = sqlite3.connect(caminho_pasta+bd)
        cursor = conn.cursor()

        cursor.execute(query)
        result = cursor.fetchall()

        if result:
            columns = [description[0] for description in cursor.description]
            # metadata = "| " + " | ".join(columns) + " |"
            # print(metadata)
            # print("-" * (len(metadata)))
            count=0
            for row in result:
              print('\nLinha', str(count),':')
              i = 0
              for item in row:
                print(columns[i],'=',str(item))
                i+=1
              count+=1

        else:
            print("Nenhum resultado encontrado =(")

        cursor.close()
        conn.close()

    except sqlite3.Error as e:
        print("Ocorreu um erro ao executar a consulta:", e)

In [8]:
def alterar(bd, query):
    try:
        conn = sqlite3.connect(caminho_pasta+bd)
        cursor = conn.cursor()

        cursor.execute(query)
        conn.commit()

        print("Alterações no banco de dados realizadas com sucesso!")

        cursor.close()
        conn.close()

    except sqlite3.Error as e:
        print("Ocorreu um erro ao executar a alteração no banco de dados:", e)

In [9]:
def renomearColunas(bd, nome_tabela, mapeamento_colunas):
    try:
        conn = sqlite3.connect(caminho_pasta+bd)
        cursor = conn.cursor()

        for nome_antigo, nome_novo in mapeamento_colunas.items():
            query = f"ALTER TABLE {nome_tabela} RENAME COLUMN {nome_antigo} TO {nome_novo};"
            cursor.execute(query)

        conn.commit()

        cursor.close()
        conn.close()

        print("Nomes das colunas alterados com sucesso!")

    except sqlite3.Error as e:
        print("Ocorreu um erro ao alterar os nomes das colunas:", e)

### Inglês

In [ ]:
db_name_EN = 'sqliteEN-sample.db'

In [ ]:
db_path_EN = caminho_pasta + db_name_EN

In [ ]:
informacoes(db_name_EN)


Table: country- 255 linhas.
* name - TEXT   (PK= False )
* code - INT   (PK= False )

Table: company- 28554319 linhas.
* basic_cnpj - TEXT   (PK= False )
* name - TEXT   (PK= False )
* responsible_federal_entity - TEXT   (PK= False )
* capital - DOUBLE   (PK= False )
* legal_nature_code - INT   (PK= False )
* responsible_qualification_code - INT   (PK= False )
* company_size_code - INT   (PK= False )

Table: economic_activity- 1359 linhas.
* name - TEXT   (PK= False )
* code - INT   (PK= False )

Table: registration_status_reason- 61 linhas.
* description - TEXT   (PK= False )
* code - INT   (PK= False )

Table: city- 5571 linhas.
* name - TEXT   (PK= False )
* code - INT   (PK= False )

Table: legal_nature- 90 linhas.
* description - TEXT   (PK= False )
* code - INT   (PK= False )

Table: qualification- 68 linhas.
* description - TEXT   (PK= False )
* code - INT   (PK= False )

Table: establishment- 29999992 linhas.
* basic_cnpj - TEXT   (PK= False )
* order_cnpj - TEXT   (PK= False 

#### Chaves Primárias e Estrangeiras

##### City

In [ ]:
query_create_city = """
CREATE TABLE city_new (
  code INTEGER PRIMARY KEY,
  name TEXT
);
"""
alterar(db_name_EN,query_create_city)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO city_new (code, name) SELECT code, name FROM city;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE city;")
alterar(db_name_EN,"ALTER TABLE city_new RENAME TO city;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Country

In [ ]:
query_create_country = """
CREATE TABLE country_new (
  code INTEGER PRIMARY KEY,
  name TEXT
);
"""
alterar(db_name_EN,query_create_country)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO country_new (code, name) SELECT code, name FROM country;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE country;")
alterar(db_name_EN,"ALTER TABLE country_new RENAME TO country;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Economic Activity

In [ ]:
query_create_economic_activity = """
CREATE TABLE cnae (
  code INTEGER PRIMARY KEY,
  name TEXT
);
"""
alterar(db_name_EN,query_create_economic_activity)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO cnae (code, name) SELECT code, name FROM economic_activity;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE economic_activity;")

Alterações no banco de dados realizadas com sucesso!


##### Registration Status Reason

In [ ]:
query_create_registration_status_reason = """
CREATE TABLE registration_status_reason_new (
  code INTEGER PRIMARY KEY,
  description TEXT
);
"""
alterar(db_name_EN,query_create_registration_status_reason)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO registration_status_reason_new (code, description) SELECT code, description FROM registration_status_reason;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE registration_status_reason;")
alterar(db_name_EN,"ALTER TABLE registration_status_reason_new RENAME TO registration_status_reason;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Legal Nature

In [ ]:
query_create_legal_nature = """
CREATE TABLE legal_nature_new (
  code INTEGER PRIMARY KEY,
  description TEXT
);
"""
alterar(db_name_EN,query_create_legal_nature)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO legal_nature_new (code, description) SELECT code, description FROM legal_nature;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE legal_nature;")
alterar(db_name_EN,"ALTER TABLE legal_nature_new RENAME TO legal_nature;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Qualification

In [ ]:
query_create_qualification = """
CREATE TABLE qualification_new (
  code INTEGER PRIMARY KEY,
  description TEXT
);
"""
alterar(db_name_EN,query_create_qualification)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO qualification_new (code, description) SELECT code, description FROM qualification;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE qualification;")
alterar(db_name_EN,"ALTER TABLE qualification_new RENAME TO qualification;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Establishment

In [ ]:
alterar(db_name_EN,"CREATE TABLE establishment_limited AS SELECT * FROM establishment WHERE (basic_cnpj, order_cnpj, cnpj_verification_digit) IN (SELECT DISTINCT basic_cnpj, order_cnpj, cnpj_verification_digit FROM establishment LIMIT 100000);")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
consultar(db_name_EN, "SELECT COUNT(DISTINCT (basic_cnpj || order_cnpj || cnpj_verification_digit)) FROM establishment_limited;")


Linha 0 :
COUNT(DISTINCT (basic_cnpj || order_cnpj || cnpj_verification_digit)) = 100000


In [ ]:
alterar(db_name_EN,"DROP TABLE establishment")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
query_create_establishment = """
CREATE TABLE establishment (
  basic_cnpj TEXT,
  order_cnpj TEXT,
  cnpj_verification_digit TEXT,
  main_or_branch TEXT,
  name TEXT,
  start_activity_date DATE,
  primary_cnae_code INTEGER,
  secondary_cnae_code TEXT,
  city_code INTEGER,
  state TEXT,
  country_code INTEGER,
  foreign_city_name TEXT,
  special_status TEXT,
  registration_status_code INTEGER,
  registration_status_reason_code INTEGER,

  PRIMARY KEY (basic_cnpj,order_cnpj,cnpj_verification_digit),
  FOREIGN KEY (primary_cnae_code) REFERENCES cnae(code),
  FOREIGN KEY (city_code) REFERENCES city(code),
  FOREIGN KEY (country_code) REFERENCES country(code)
  FOREIGN KEY (registration_status_code) REFERENCES registration_status(code)
  FOREIGN KEY (registration_status_reason_code) REFERENCES registration_status_reason(code)
);
"""
alterar(db_name_EN,query_create_establishment)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO establishment (basic_cnpj, order_cnpj, cnpj_verification_digit, main_or_branch, name, start_activity_date, primary_cnae_code, secondary_cnae_code, city_code, state, country_code, foreign_city_name, special_status, registration_status_code, registration_status_reason_code) SELECT basic_cnpj, order_cnpj, cnpj_verification_digit, main_or_branch,name,start_activity_date,primary_economic_activity_code,secondary_economic_activity_code,city_code,state, country_code, foreign_city_name, special_status, registration_status_code,registration_status_reason_code FROM establishment_limited;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE establishment_limited;")

Alterações no banco de dados realizadas com sucesso!


##### Company

In [ ]:
consultar(db_name_EN,"SELECT COUNT(DISTINCT basic_cnpj) FROM company;")


Linha 0 :
COUNT(DISTINCT basic_cnpj) = 24551470


In [ ]:
alterar(db_name_EN,"CREATE TABLE company_distinct AS SELECT DISTINCT basic_cnpj, name, capital, responsible_federal_entity, legal_nature_code, responsible_qualification_code, company_size_code FROM company;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
consultar(db_name_EN,"SELECT COUNT(DISTINCT basic_cnpj) FROM company_distinct;")


Linha 0 :
COUNT(DISTINCT basic_cnpj) = 24551470


In [ ]:
consultar(db_name_EN,"SELECT basic_cnpj, COUNT(*) FROM company_distinct GROUP BY basic_cnpj HAVING COUNT(*) > 1 ")


Linha 0 :
basic_cnpj = 08725575
COUNT(*) = 2


In [ ]:
consultar(db_name_EN,"SELECT basic_cnpj FROM establishment WHERE basic_cnpj = '08725575'")

Nenhum resultado encontrado =(


In [ ]:
alterar(db_name_EN,"DROP TABLE company")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
query_create_company = """
CREATE TABLE company_new (
  basic_cnpj TEXT PRIMARY KEY,
  name TEXT,
  capital DOUBLE,
  responsible_federal_entity TEXT,
  legal_nature_code INTEGER,
  responsible_qualification_code INTEGER,
  company_size_code INTEGER,

  FOREIGN KEY (basic_cnpj) REFERENCES establishment(basic_cnpj),
  FOREIGN KEY (legal_nature_code) REFERENCES legal_nature(code),
  FOREIGN KEY (responsible_qualification_code) REFERENCES qualification(code),
  FOREIGN KEY (company_size_code) REFERENCES company_size(code)
);
"""
alterar(db_name_EN,query_create_company)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO company_new (basic_cnpj, name, capital, responsible_federal_entity, legal_nature_code, responsible_qualification_code, company_size_code) SELECT basic_cnpj, name, capital, responsible_federal_entity, legal_nature_code, responsible_qualification_code, company_size_code FROM company_distinct WHERE basic_cnpj IN (SELECT basic_cnpj FROM establishment);")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE company_distinct;")
alterar(db_name_EN,"ALTER TABLE company_new RENAME TO company;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Partner

In [ ]:
query_create_partner = """
CREATE TABLE partner_new (
  basic_cnpj TEXT,
  name TEXT,
  cpf_or_cnpj TEXT,
  legal_representative_cpf TEXT,
  legal_representative_name TEXT,
  partner_type_code INTEGER,
  partner_qualification_code INTEGER,
  country_code INTEGER,
  legal_representative_qualification_code INTEGER,
  age_range_code INTEGER,

  FOREIGN KEY (basic_cnpj) REFERENCES establishment(basic_cnpj),
  FOREIGN KEY (partner_type_code) REFERENCES partner_type(code),
  FOREIGN KEY (partner_qualification_code) REFERENCES qualification(code),
  FOREIGN KEY (legal_representative_qualification_code) REFERENCES qualification(code),
  FOREIGN KEY (country_code) REFERENCES country(code)
  FOREIGN KEY (age_range_code) REFERENCES age_range(code)
);
"""
alterar(db_name_EN,query_create_partner)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO partner_new (basic_cnpj, name, cpf_or_cnpj, legal_representative_cpf, legal_representative_name, partner_type_code, partner_qualification_code, country_code, legal_representative_qualification_code, age_range_code) SELECT basic_cnpj, name, cpf_or_cnpj, legal_representative_cpf, legal_representative_name, partner_type_code, partner_qualification_code, country_code, legal_representative_qualification_code, age_range_code FROM partner WHERE basic_cnpj IN (SELECT basic_cnpj FROM establishment);")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE partner;")
alterar(db_name_EN,"ALTER TABLE partner_new RENAME TO partner;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Taxation

In [ ]:
consultar(db_name_EN,"SELECT COUNT(DISTINCT basic_cnpj) FROM taxation;")


Linha 0 :
COUNT(DISTINCT basic_cnpj) = 18977400


In [ ]:
alterar(db_name_EN,"CREATE TABLE taxation_distinct AS SELECT DISTINCT basic_cnpj,option_for_simples_taxation,option_for_mei_taxation,simples_taxation_option_date,simples_taxation_exclusion_date,mei_taxation_option_date,mei_taxation_exclusion_date FROM taxation;")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
consultar(db_name_EN,"SELECT COUNT(DISTINCT basic_cnpj) FROM taxation_distinct;")


Linha 0 :
COUNT(DISTINCT basic_cnpj) = 18977400


In [ ]:
alterar(db_name_EN,"DROP TABLE taxation")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
query_create_taxation = """
CREATE TABLE taxation_new (
  basic_cnpj TEXT,
  option_for_simples_taxation CHAR,
  option_for_mei_taxation CHAR,
  simples_taxation_option_date DATE,
  simples_taxation_exclusion_date DATE,
  mei_taxation_option_date DATE,
  mei_taxation_exclusion_date DATE,

  FOREIGN KEY (basic_cnpj) REFERENCES establishment(basic_cnpj)
);
"""
alterar(db_name_EN,query_create_taxation)

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"INSERT INTO taxation_new (basic_cnpj,option_for_simples_taxation,option_for_mei_taxation,simples_taxation_option_date,simples_taxation_exclusion_date,mei_taxation_option_date,mei_taxation_exclusion_date) SELECT basic_cnpj,option_for_simples_taxation,option_for_mei_taxation,simples_taxation_option_date,simples_taxation_exclusion_date,mei_taxation_option_date,mei_taxation_exclusion_date FROM taxation_distinct WHERE basic_cnpj IN (SELECT basic_cnpj FROM establishment);")

Alterações no banco de dados realizadas com sucesso!


In [ ]:
alterar(db_name_EN,"DROP TABLE taxation_distinct;")
alterar(db_name_EN,"ALTER TABLE taxation_new RENAME TO taxation;")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


#### Resultado Final

In [ ]:
informacoes(db_name_EN)


Table: company_size- 4 linhas.
* code - INT   (PK= True )
* description - TEXT   (PK= False )

Table: registration_status- 5 linhas.
* code - INT   (PK= True )
* description - TEXT   (PK= False )

Table: partner_type- 3 linhas.
* code - INT   (PK= True )
* description - TEXT   (PK= False )

Table: age_range- 10 linhas.
* code - INT   (PK= True )
* description - TEXT   (PK= False )

Table: city- 5571 linhas.
* code - INTEGER   (PK= True )
* name - TEXT   (PK= False )

Table: country- 255 linhas.
* code - INTEGER   (PK= True )
* name - TEXT   (PK= False )

Table: cnae- 1359 linhas.
* code - INTEGER   (PK= True )
* name - TEXT   (PK= False )

Table: registration_status_reason- 61 linhas.
* code - INTEGER   (PK= True )
* description - TEXT   (PK= False )

Table: legal_nature- 90 linhas.
* code - INTEGER   (PK= True )
* description - TEXT   (PK= False )

Table: qualification- 68 linhas.
* code - INTEGER   (PK= True )
* description - TEXT   (PK= False )

Table: establishment- 100000 linhas.

In [ ]:
db_EN = get_schema(db_path_EN)
print(db_EN)


CREATE TABLE age_range (
	code INTEGER, 
	description TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE city (
	code INTEGER, 
	name TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE cnae (
	code INTEGER, 
	name TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE company (
	basic_cnpj TEXT, 
	name TEXT, 
	capital DOUBLE, 
	responsible_federal_entity TEXT, 
	legal_nature_code INTEGER, 
	responsible_qualification_code INTEGER, 
	company_size_code INTEGER, 
	PRIMARY KEY (basic_cnpj), 
	FOREIGN KEY(basic_cnpj) REFERENCES establishment (basic_cnpj), 
	FOREIGN KEY(legal_nature_code) REFERENCES legal_nature (code), 
	FOREIGN KEY(responsible_qualification_code) REFERENCES qualification (code), 
	FOREIGN KEY(company_size_code) REFERENCES company_size (code)
)


CREATE TABLE company_size (
	code INTEGER, 
	description TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE country (
	code INTEGER, 
	name TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE establishment (
	basic_cnpj TEXT, 
	order_cnpj TEXT, 
	cnpj_verification_digit 

### Português -> Renomear

In [10]:
db_name_PT = 'sqlitePT-sample.db'

In [11]:
db_path_PT = caminho_pasta + db_name_PT

In [ ]:
db_PT = get_schema(db_path_PT)
print(db_PT)


CREATE TABLE age_range (
	code INTEGER, 
	description TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE city (
	code INTEGER, 
	name TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE cnae (
	code INTEGER, 
	name TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE company (
	basic_cnpj TEXT, 
	name TEXT, 
	capital DOUBLE, 
	responsible_federal_entity TEXT, 
	legal_nature_code INTEGER, 
	responsible_qualification_code INTEGER, 
	company_size_code INTEGER, 
	PRIMARY KEY (basic_cnpj), 
	FOREIGN KEY(basic_cnpj) REFERENCES establishment (basic_cnpj), 
	FOREIGN KEY(legal_nature_code) REFERENCES legal_nature (code), 
	FOREIGN KEY(responsible_qualification_code) REFERENCES qualification (code), 
	FOREIGN KEY(company_size_code) REFERENCES company_size (code)
)


CREATE TABLE company_size (
	code INTEGER, 
	description TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE country (
	code INTEGER, 
	name TEXT, 
	PRIMARY KEY (code)
)


CREATE TABLE establishment (
	basic_cnpj TEXT, 
	order_cnpj TEXT, 
	cnpj_verification_digit 

##### Name->Nome

In [ ]:
traducao_nome = {
  'code':'codigo',
	'name':'nome',
}

###### cidade

In [ ]:
alterar(db_name_PT,'ALTER TABLE city RENAME TO cidade')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
renomearColunas(db_name_PT,'cidade', traducao_nome )

Nomes das colunas alterados com sucesso!


###### pais

In [ ]:
alterar(db_name_PT,'ALTER TABLE country RENAME TO pais')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
renomearColunas(db_name_PT,'pais', traducao_nome )

Nomes das colunas alterados com sucesso!


###### cnae

In [ ]:
renomearColunas(db_name_PT,'cnae', traducao_nome )

Nomes das colunas alterados com sucesso!


##### Description->Descricao

In [ ]:
traducao_descricao = {
  'code':'codigo',
	'description':'descricao',
}

###### faixa etaria

In [ ]:
alterar(db_name_PT,'ALTER TABLE age_range RENAME TO faixa_etaria')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
renomearColunas(db_name_PT,'faixa_etaria', traducao_descricao )

Nomes das colunas alterados com sucesso!


###### porte empresa

In [ ]:
alterar(db_name_PT,'ALTER TABLE company_size RENAME TO porte_empresa')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
renomearColunas(db_name_PT,'porte_empresa', traducao_descricao )

Nomes das colunas alterados com sucesso!


###### natureza juridica

In [ ]:
alterar(db_name_PT,'ALTER TABLE legal_nature RENAME TO natureza_juridica')
renomearColunas(db_name_PT,'natureza_juridica', traducao_descricao )

Alterações no banco de dados realizadas com sucesso!
Nomes das colunas alterados com sucesso!


###### tipo socio

In [ ]:
alterar(db_name_PT,'ALTER TABLE partner_type RENAME TO tipo_socio')
renomearColunas(db_name_PT,'tipo_socio', traducao_descricao)

Alterações no banco de dados realizadas com sucesso!
Nomes das colunas alterados com sucesso!


###### qualificacao

In [ ]:
alterar(db_name_PT,'ALTER TABLE qualification RENAME TO qualificacao')
renomearColunas(db_name_PT,'qualificacao', traducao_descricao)

Alterações no banco de dados realizadas com sucesso!
Nomes das colunas alterados com sucesso!


###### situacao cadastral

In [ ]:
alterar(db_name_PT,'ALTER TABLE registration_status RENAME TO situacao_cadastral')
renomearColunas(db_name_PT,'situacao_cadastral', traducao_descricao)

Alterações no banco de dados realizadas com sucesso!
Nomes das colunas alterados com sucesso!


###### motivo situacao cadastral

In [ ]:
alterar(db_name_PT,'ALTER TABLE registration_status_reason RENAME TO motivo_situacao_cadastral')
renomearColunas(db_name_PT,'motivo_situacao_cadastral', traducao_descricao)

Alterações no banco de dados realizadas com sucesso!
Nomes das colunas alterados com sucesso!


##### Company -> Empresa

In [ ]:
alterar(db_name_PT,'ALTER TABLE company RENAME TO empresa')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
traducao_colunas_empresa = {
  'basic_cnpj':'cnpj_basico',
	'name':'nome',
	'capital':'capital',
	'responsible_federal_entity':'ente_federativo_responsavel',
	'legal_nature_code':'codigo_natureza_juridica',
	'responsible_qualification_code':'codigo_qualificacao_responsavel',
	'company_size_code':'codigo_porte_empresa'
}

In [ ]:
renomearColunas(db_name_PT,'empresa', traducao_colunas_empresa )

Nomes das colunas alterados com sucesso!


##### Establishment -> Estabelecimento

In [ ]:
alterar(db_name_PT,'ALTER TABLE establishment RENAME TO estabelecimento')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
traducao_colunas_estabelecimento = {
  'basic_cnpj':'cnpj_basico',
	'order_cnpj':'cnpj_ordem',
	'cnpj_verification_digit':'cnpj_digito_verificador',
	'main_or_branch':'matriz_ou_filial',
	'name':'nome',
	'start_activity_date':'data_inicio_atividade',
	'primary_cnae_code':'codigo_cnae_principal',
	'secondary_cnae_code':'codigo_cnae_secundaria',
	'city_code':'codigo_cidade',
	'state':'estado',
	'country_code':'codigo_pais',
	'foreign_city_name':'nome_cidade_estrangeira',
	'special_status':'situacao_especial',
	'registration_status_code':'codigo_situacao_cadastral',
	'registration_status_reason_code':'codigo_motivo_situacao_cadastral',
}

In [ ]:
renomearColunas(db_name_PT,'estabelecimento', traducao_colunas_estabelecimento )

Nomes das colunas alterados com sucesso!


In [ ]:
alterar(db_name_PT, "UPDATE estabelecimento SET matriz_ou_filial = 'Matriz' WHERE matriz_ou_filial = 'Main'")
alterar(db_name_PT, "UPDATE estabelecimento SET matriz_ou_filial = 'Filial' WHERE matriz_ou_filial = 'Branch'")

Alterações no banco de dados realizadas com sucesso!
Alterações no banco de dados realizadas com sucesso!


##### Partner -> Socio

In [ ]:
alterar(db_name_PT,'ALTER TABLE partner RENAME TO socio')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
traducao_colunas_socio = {
  'basic_cnpj':'cnpj_basico',
	'name':'nome',
	'cpf_or_cnpj':'cpf_ou_cnpj',
	'legal_representative_cpf':'cpf_representante_legal',
	'legal_representative_name':'nome_representante_legal',
	'partner_type_code':'codigo_tipo_socio',
	'partner_qualification_code':'codigo_qualificacao_socio',
	'country_code':'codigo_pais',
	'legal_representative_qualification_code':'codigo_qualificacao_representante_legal',
	'age_range_code':'codigo_faixa_etaria'
}

In [ ]:
renomearColunas(db_name_PT,'socio', traducao_colunas_socio )

Nomes das colunas alterados com sucesso!


##### Taxation -> Tributacao

In [ ]:
alterar(db_name_PT,'ALTER TABLE taxation RENAME TO tributacao')

Alterações no banco de dados realizadas com sucesso!


In [ ]:
traducao_colunas_tributacao = {
  'basic_cnpj':'cnpj_basico',
	'option_for_simples_taxation':'opcao_pelo_simples_nacional',
	'option_for_mei_taxation':'opcao_pelo_mei',
	'simples_taxation_option_date':'data_opcao_simples_nacional',
	'simples_taxation_exclusion_date':'data_exclusao_simples_nacional',
	'mei_taxation_option_date':'data_opcao_mei',
	'mei_taxation_exclusion_date':'data_exclusao_mei',
}

In [ ]:
renomearColunas(db_name_PT,'tributacao', traducao_colunas_tributacao )

Nomes das colunas alterados com sucesso!


#### Resultado Final

In [15]:
db_PT = get_schema(db_path_PT)
print(db_PT)


CREATE TABLE cidade (
	codigo INTEGER, 
	nome TEXT, 
	PRIMARY KEY (codigo)
)


CREATE TABLE cnae (
	codigo INTEGER, 
	nome TEXT, 
	PRIMARY KEY (codigo)
)


CREATE TABLE empresa (
	cnpj_basico TEXT, 
	nome TEXT, 
	capital DOUBLE, 
	ente_federativo_responsavel TEXT, 
	codigo_natureza_juridica INTEGER, 
	codigo_qualificacao_responsavel INTEGER, 
	codigo_porte_empresa INTEGER, 
	PRIMARY KEY (cnpj_basico), 
	FOREIGN KEY(cnpj_basico) REFERENCES estabelecimento (cnpj_basico), 
	FOREIGN KEY(codigo_natureza_juridica) REFERENCES natureza_juridica (codigo), 
	FOREIGN KEY(codigo_qualificacao_responsavel) REFERENCES qualificacao (codigo), 
	FOREIGN KEY(codigo_porte_empresa) REFERENCES porte_empresa (codigo)
)


CREATE TABLE estabelecimento (
	cnpj_basico TEXT, 
	cnpj_ordem TEXT, 
	cnpj_digito_verificador TEXT, 
	matriz_ou_filial TEXT, 
	nome TEXT, 
	data_inicio_atividade DATE, 
	codigo_cnae_principal INTEGER, 
	codigo_cnae_secundaria TEXT, 
	codigo_cidade INTEGER, 
	estado TEXT, 
	codigo_pais INTE